In [1]:
from tqdm import tqdm
import torch
from torch import Tensor
import numpy as np
from jaxtyping import Int, Float
from typing import List, Optional, Tuple
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
from tqdm import tqdm
from einops import repeat
import multiprocessing
import sys

from DSA.dmd import DMD
from DSA.simdist import SimilarityTransformDist
from sklearn.manifold import MDS

sys.path.append('../')
from models import get_model_from_name
from data import get_wikitext_103_sample
from activations import get_residual_stream_activations_for_layer_wise_dmd
from dmd import apply_dmd

torch.set_grad_enabled(True)

# List all available GPUs
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")

else:
    print("CUDA is not available. Listing CPUs instead.")
    print(multiprocessing.cpu_count())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/jsalle/.conda/envs/p11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA is not available. Listing CPUs instead.
48


In [3]:
models_names = ['gpt2-small', 'gpt2-medium', 'pythia-70m', 'pythia-1b', 'pythia-2.8b', 'bert-base-uncased']
models = []
model_types = []

for i, model_name in enumerate(models_names):
    models, model_types = get_residual_stream_activations_for_layer_wise_dmd(model_name,
                                                                             models,
                                                                             model_types)

sims_dmd, model_types, mds_df = apply_dmd(models, 
                                  model_types, 
                                  device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Computing Hankel matrix ...
Hankel matrix computed!
Computing SVD on Hankel matrix ...
H torch.Size([1015, 7680])
SVD complete!
Computing least squares fits to HAVOK DMD ...
Least squares complete! 

Computing Hankel matrix ...
Hankel matrix computed!
Computing SVD on Hankel matrix ...
H torch.Size([1015, 7680])
SVD complete!
Computing least squares fits to HAVOK DMD ...
Least squares complete! 

Computing Hankel matrix ...
Hankel matrix computed!
Computing SVD on Hankel matrix ...
H torch.Size([1015, 7680])
SVD complete!
Computing least squares fits to HAVOK DMD ...
Least squares complete! 

Computing Hankel matrix ...
Hankel matrix computed!
Computing SVD on Hankel matrix ...
H torch.Size([1015, 7680])
SVD complete!
Computing least squares fits to HAVOK DMD ...
Least squares complete! 

Computing Hankel matrix ...
Hankel matrix computed!
Computing SVD on Hankel matrix ...
H torch.Size([1015, 7680])
SVD complete!
Computing least squares fits to HAVOK DMD ...
Least squares complete! 



In [47]:
architectures = []
layers = []

for index, row in mds_df.iterrows():
    architectures.append(row['Model Type'].rsplit(' -')[0])
    layers.append(int(row['Model Type'].rsplit('layer ')[1]))

mds_df['Architecture'] = architectures
mds_df['Layer'] = layers
mds_df

,Model Type,DMD:0,DMD:1,Architecture,Layer
0,pythia-14m - layer 0,0.142414,-0.377293,pythia-14m,0
1,pythia-14m - layer 1,0.096191,-0.113576,pythia-14m,1
2,pythia-14m - layer 2,-0.154431,0.053843,pythia-14m,2
3,pythia-14m - layer 3,-0.247504,0.126716,pythia-14m,3
4,pythia-14m - layer 4,-0.182962,0.470755,pythia-14m,4
5,pythia-14m - layer 5,0.346292,-0.160446,pythia-14m,5


In [37]:
import plotly.express as px

fig = px.scatter(mds_df, 
                 x="DMD:0", 
                 y="DMD:1", 
                 color="Architecture",
                 title="",
                 labels={
                     "DMD:0": "DMD (dim 0)",
                     "DMD:1": "DMD (dim 1)",
                     "Layer": "Layer"
                 },)
fig.show()

In [48]:
fig = px.scatter(mds_df, x="DMD:0", y="DMD:1", color='Layer', symbol='Architecture')
fig.show()